In [2]:
from music21 import *
import music21
# music21.environment.set("musicxmlPath", "/usr/bin/musescore")
import pandas as pd
import json

In [3]:
def info_Chord(note, code, code_li, tt_li, number, qq, expression_bool):
    duration = float(note.quarterLength) # 박자수
    if duration>0:
#         continue
        d_type = note.duration.fullName # 음표종류

        #                             d_type = note.duration.type # 음표종류
        #                 print(note.duration.fullName)
        offset = float(note.offset)
        #                             velocity = note.volume.realized
        for chord_note in note.pitches:
            name = chord_note.name # 음이름
            octave = chord_note.octave # 옥타브
            pitch = chord_note.ps
            
            if expression_bool:
                code_name = str(code).split('Expression')[-1][1:-1]
                if '\'' in code_name:
                    code_name = code_name.replace('\'', '')
#             elif 'NoChord' in str(code):
#                 code_name = str(code).split('NoChord')[-1][1:-1]
            else:
                code_name = str(code).split('ChordSymbol')[-1][1:-1]
                if '\'' in code_name:
                    code_name = code_name.replace('\'', '')
        #                                 print(code_name)
            if not code_name in code_li:
                code_li.append(code_name)

            while tt_li[number-1][qq] != [[]] and code_name!=tt_li[number-1][qq][0]:
#                 if tt_li[number-1][qq+1] != [[]] and code_name!=tt_li[number-1][qq+1][0]:
                tt_li[number-1].append([[]])
                qq+=1
                
                
            tt_li[number-1][qq][0]=code_name
            tt_li[number-1][qq].append({'pitch':pitch, 'name':name, 'octave':octave, 'position':offset,'d_type':d_type, 'duration':float(duration)})
    return tt_li, code_li

def info_Rest(note, code, code_li, tt_li, number, qq, expression_bool):
    duration = float(note.quarterLength)
    if duration>0:
        d_type = note.duration.fullName

    #                             d_type = note.duration.type
        name = note.name
        offset = float(note.offset)
        
        if expression_bool:
                code_name = str(code).split('Expression')[-1][1:-1]
                if '\'' in code_name:
                    code_name = code_name.replace('\'', '')
#         elif 'NoChord' in str(code):
#             code_name = str(code).split('NoChord')[-1][1:-1]
        else:
            code_name = str(code).split('ChordSymbol')[-1][1:-1]
            if '\'' in code_name:
                code_name = code_name.replace('\'', '')
    #                             print(code_name)
        if not code_name in code_li:
            code_li.append(code_name)

        while tt_li[number-1][qq] != [[]] and code_name!=tt_li[number-1][qq][0]:
#                 if tt_li[number-1][qq+1] != [[]] and code_name!=tt_li[number-1][qq+1][0]:
            tt_li[number-1].append([[]])
            qq+=1
    #                                 print('add tt', tt_li[number-1][qq])

        tt_li[number-1][qq][0]=code_name
        tt_li[number-1][qq].append({'name':name, 'position':offset,'d_type':d_type, 'duration':float(duration)})

    return tt_li, code_li

def info_Note(note, code, code_li, tt_li, number, qq, expression_bool):    
    duration = float(note.quarterLength) 
    
    if duration>0.0:
#         continue
        pitch = note.pitch.ps
        name = note.name # 음이름
        octave = note.octave # 옥타브
        d_type = note.duration.type # 음표종류
        d_type = note.duration.fullName
        offset = float(note.offset)
        
        
        if expression_bool:
            code_name = str(code).split('Expression')[-1][1:-1]
            if '\'' in code_name:
                code_name = code_name.replace('\'', '')
#         elif 'NoChord' in str(code):
#             code_name = str(code).split('NoChord')[-1][1:-1]
        else:
            code_name = str(code).split('ChordSymbol')[-1][1:-1]
            if '\'' in code_name:
                code_name = code_name.replace('\'', '')
    #                             print(code_name)
        if not code_name in code_li:
            code_li.append(code_name)

    #                             print(tt_li[number-1][qq])
        while tt_li[number-1][qq] != [[]] and code_name!=tt_li[number-1][qq][0]:
            tt_li[number-1].append([[]])
            qq+=1
    #                                 print('add tt', tt_li[number-1][qq])

        tt_li[number-1][qq][0]=code_name
        tt_li[number-1][qq].append({'pitch':pitch, 'name':name, 'octave':octave, 'position':offset,'d_type':d_type, 'duration':float(duration)})
        
    return tt_li, code_li
#     else:
#         print(note.show('txt'), note.quarterLength)
    

In [4]:
def extract_data(part, x, ii, tt_li, code_li, expression_bool):
    k=0
    for note in part.flat.notesAndRests:
        qq=0
        if expression_bool:
            
            if k<len(xml_data.parts[x].flat.getElementsByClass('Expression'))-1:
                k+=1
            note1 = xml_data.parts[x].flat.getElementsByClass('Expression')[k]
        else:
            note1 = xml_data.parts[x].flat.notesAndRests.getElementsByClass('Harmony')[k]
        
        number=note.measureNumber
        code_number=note1.measureNumber

        note_offset = note.offset
        
        if expression_bool:
            code_offset = xml_data.parts[x].flat.getElementsByClass('Expression')[k].offset
        else:
            code_offset = xml_data.parts[x].flat.notesAndRests.getElementsByClass('Harmony')[k].offset
        
        if k<len(xml_data.parts[x].flat.getElementsByClass('Expression'))-1:
            next_code_offset = xml_data.parts[x].flat.getElementsByClass('Expression')[k+1].offset
            
        if k<len(xml_data.parts[x].flat.notesAndRests.getElementsByClass('Harmony'))-1:
            next_code_offset = xml_data.parts[x].flat.notesAndRests.getElementsByClass('Harmony')[k+1].offset

        
        if note_offset>0 and note_offset>=next_code_offset:
            if expression_bool:
                if k<len(xml_data.parts[x].flat.getElementsByClass('Expression'))-1:
                    k+=1

                note1 = xml_data.parts[x].flat.getElementsByClass('Expression')[k]

            else:
                if k<len(xml_data.parts[x].flat.notesAndRests.getElementsByClass('Harmony'))-1:
                    k+=1
                if not xml_data.parts[x].flat.notesAndRests.getElementsByClass('Harmony')[k].pitches and k<len(xml_data.parts[x].flat.notesAndRests.getElementsByClass('Harmony'))-1:
                    k+=1

                note1 = xml_data.parts[x].flat.notesAndRests.getElementsByClass('Harmony')[k]
        
#         print(note_offset, next_code_offset, note1)
        if ii==0:
            code = note1
            if 'NoChord' in str(code):
                continue
#             print('info pm : ',note, code, code_li, tt_li, number, qq)
#                         print(number, code_number, code, code.pitches,  note.offset, note1.offset)
            if note.isChord:
                tt_li, code_li = info_Chord(note, code, code_li, tt_li, number, qq, expression_bool)

            elif note.isRest:
                tt_li, code_li = info_Rest(note, code, code_li, tt_li, number, qq, expression_bool)

            else:
                tt_li, code_li = info_Note(note, code, code_li, tt_li, number, qq, expression_bool)
                
        else:    
            if number>=code_number:
                if number >= code_number:
                    code = note1
                
                if 'NoChord' in str(code):
                    continue
                
                if note.isChord:
                    tt_li, code_li = info_Chord(note, code, code_li, tt_li,number, qq, expression_bool)
                    
                elif note.isRest:
                    tt_li, code_li = info_Rest(note, code, code_li, tt_li,number, qq, expression_bool)

                else:
                    tt_li, code_li = info_Note(note, code, code_li, tt_li,number, qq, expression_bool)

        if number==0:
            k+=1
            
    return tt_li, code_li

In [5]:
def xml_to_list(xml, expression_bool):
    """Convert a music xml file to a list of note events

    Notebook: C1/C1S2_MusicXML.ipynb

    Args:
        xml (str or music21.stream.Score): Either a path to a music xml file or a music21.stream.Score

    Returns:
        score (list): A list of note events where each note is specified as
            ``[start, duration, pitch, velocity, label]``
    """

    if isinstance(xml, str):
        xml_data = converter.parse(xml)
    elif isinstance(xml, stream.Score):
        xml_data = xml
    else:
        raise RuntimeError('midi must be a path to a midi file or music21.stream.Score')
    
    print(xml_data.parts)
    
    score = []
    
    instrument = [part.getInstrument().instrumentName for part in xml_data.parts]
    
    instrument_index = [int(i) for i, ele in enumerate(instrument) if ele == 'Piano' or ele=='Klavier' 
                        or ele=='Фортепіано' or ele == '鋼琴']
    print(instrument, instrument_index)
    
    xml_data_index = [] # 코드가 있는 부분을 찾음
#     expression_bool = False
    for s in range(len(xml_data.parts)):
        if len(xml_data.parts[s].flat.notesAndRests.getElementsByClass('Harmony'))>0:
            xml_data_index.append(s)
        
        elif expression_bool == True and len(xml_data.parts[s].flat.getElementsByClass('Expression'))>0:
            xml_data_index.append(s)#print(s)
    print('expression_bool : ',expression_bool)
    
    print(xml_data_index)
    if instrument_index[-1] in xml_data_index:
        xml_data_index = [instrument_index[-1]]
    print(xml_data_index)
    
    TimeSignature = xml_data.parts[0][1].timeSignature # n/n박자
    basesignature = int(TimeSignature.denominator)
#     print(basesignature)
    
    tmp_li = list()
    number_cnt = xml_data.parts[instrument_index[-1]].flat.notesAndRests[-1].measureNumber
    tt_li = [[[[]]] for i in range(number_cnt)] # 같은 마디 정보 저장
    code_li = list()
    for i, part in enumerate(xml_data.parts):
                                              
        if i == instrument_index[-1]:
        
            instrument = part.getInstrument().instrumentName
            print(instrument)
#             print(len(part.flat.notes), len(xml_data.parts[0]))
            
#             k=0
            for ii, x in enumerate(xml_data_index):
                print('x : ', x)
                
                tt_li, code_li = extract_data(part, x, ii, tt_li, code_li, expression_bool)
                
                
    return score, json, tt_li, code_li

In [6]:
except_li=['21. Que_Sera_Sera__Lead_sheet_with_lyrics_.mxl','6. Fireflies-Owl_City.mxl']

file_name = [i for i in os.listdir('./01_data/') if not 'ipynb' in i if not i in except_li]
file_name

['10. Top_of_the_world.mxl',
 '11. What_Is_Love_-_Haddaway.mxl',
 '12. When_I_Was_Your_by_Man_Bruno_Mars_Violin__Piano.mxl',
 '13. When_You_Say_Nothing_At_All.mxl',
 '14. You_light_up_my_life_intermediate_piano.mxl',
 '15. Lily_Allen_Somewhere_Only_We_Know.mxl',
 '16. When_You_Say_Nothing_At_All_(╟╒├ó).mxl',
 '17. Olivia_Rodrigo_-_1_step_forward_3_steps_back.mxl',
 '18. Feeling_Good_-_Originally_recorded_by_Michael_Buble....mxl',
 '19. Eternal_Flame.mxl',
 '2. Blue_Moon.mxl',
 '20. What_a_Wonderful_World.mxl',
 '23. Ocean_Eyes_-_Billie_Eilish_Simpler.mxl',
 '24. FINNEAS_-_Break_My_Heart_Again.mxl',
 '25. Just_The_Way_You_Look_Tonight.mxl',
 '26. Against_all_odds.mxl',
 '27. Raindrops_Keep_Falling_On_My_Head.mxl',
 '28. Coldplay-Sky_full_of_stars.mxl',
 '29. One_More_Light_-_Linkin_Park_Easy_Piano_Version.mxl',
 '3. Count_on_me.mxl',
 '30. Musica_leggerissima_-_Colapesce_Di_Martino.mxl',
 '31. Your_Power.mxl',
 '32. Homesick.mxl',
 '33. Give_you_my_heartIU__.mxl',
 '4. Dance_Monkey.mxl'

In [7]:
expression_li = ['18. Feeling_Good_-_Originally_recorded_by_Michael_Buble....','23. Ocean_Eyes_-_Billie_Eilish_Simpler']

tt_li_all = list()
code_li_all = list()
for n in file_name:
    fn = './01_data/'+n #5,6
    name = fn[fn.rindex('/')+1:-4]
    print(name)
    
    expression_bool=False
    if name in expression_li:
        print(name)
        expression_bool = True
        
    xml_data = converter.parse(fn)
    xml_list, json_data, tt_li, code_li = xml_to_list(xml_data, expression_bool)

    # print(code_li)

    code_final = [[] for i in range(len(code_li))]

    for tt_i, tt in enumerate(tt_li):
        for ss_i, ss in enumerate(tt):
            if not ss==[[]]:
                code_index = code_li.index(ss[0])
                for st, t in enumerate(ss[1:]):
                    if st==0:
                        start_time = t['position']
                        tt_li[tt_i][ss_i][st+1]['position'] = t['position']-t['position']
                    else:
                        if start_time!=t['position']:
                            tt_li[tt_i][ss_i][st+1]['position'] = t['position']-start_time
                        else:
                            tt_li[tt_i][ss_i][st+1]['position'] = t['position']-t['position']

    tt_li_all.append(tt_li)
    code_li_all.append(code_li)
    
    code_final = [[] for i in range(len(code_li))]
    for tt in tt_li:
    #     print(tt)
        for ss in tt:
            if not ss==[[]]:
    #             print(ss)
                code_index = code_li.index(ss[0])
            #         print(code_index)
                code_final[code_index].append(ss[1:])

    json_data = dict()
    for code_n, num in zip(code_li, code_final):
        json_data[code_n]=num

    json_output_filename = './result/{}.json'.format(name)
    fd = open(json_output_filename, 'wt')
    json.dump(json_data, fd, indent=4)

    fd.close()


code_all = list()
for code in code_li_all:
    for c in code:
        if not c in code_all:
            code_all.append(c)
            
code_all.sort()
code_final_all = [[] for i in range(len(code_all))]
for tt_li in tt_li_all:
    for tt in tt_li:
    #     print(tt)
        for ss in tt:
            if not ss==[[]]:
    #             print(ss)
                code_index = code_all.index(ss[0])
            #         print(code_index)
                code_final_all[code_index].append(ss[1:])

json_data_all = dict()
for code_n, num in zip(code_all, code_final_all):
    json_data_all[code_n]=num

json_output_filename = './result/data_all.json'.format(name)
fd = open(json_output_filename, 'wt')
json.dump(json_data_all, fd, indent=4)
fd.close()
    # df = pd.DataFrame(xml_list, columns=['index','number','code','offset', 'pitch', 'name', 'octave', 'type', 'duration', 'velocity'])
    # print(df)
    # df.to_csv('{}.csv'.format(name), index=False)
    # c.show('musicXML')

10. Top_of_the_world
<music21.stream.iterator.StreamIterator for Score:0x7f27ed2f6400 @:0>
['Gitaar (vioolsleutel)', 'Piano', 'Piano'] [1, 2]
expression_bool :  False
[0, 1]
[0, 1]
Piano
x :  0
x :  1
11. What_Is_Love_-_Haddaway
<music21.stream.iterator.StreamIterator for Score:0x7f27ebfbe0f0 @:0>
['Melodica', 'Piano', 'Piano', 'Piano', 'Piano', 'Piano'] [1, 2, 3, 4, 5]
expression_bool :  False
[0]
[0]
Piano
x :  0
12. When_I_Was_Your_by_Man_Bruno_Mars_Violin__Piano
<music21.stream.iterator.StreamIterator for Score:0x7f27ead0e160 @:0>
['Violin', 'Piano', 'Piano'] [1, 2]
expression_bool :  False
[1, 2]
[2]
Piano
x :  2
13. When_You_Say_Nothing_At_All
<music21.stream.iterator.StreamIterator for Score:0x7f27ecc1d358 @:0>
['Tenor', 'Klavier', 'Klavier'] [1, 2]
expression_bool :  False
[0]
[0]
Klavier
x :  0
14. You_light_up_my_life_intermediate_piano
<music21.stream.iterator.StreamIterator for Score:0x7f27ea8581d0 @:0>
['Сопрано', 'Фортепіано', 'Фортепіано'] [1, 2]
expression_bool :  False